## **Fake News Classifier Using RNN**

If your dataset is not too complex or doesn’t have many long-range dependencies, a simple RNN may perform as well or better than LSTMs/Bi-LSTMs, which are designed to handle more complex patterns. LSTMs shine when the task requires learning long-term dependencies, but if your data has short or clear relationships, a simple RNN may work just fine.

In [2]:
import pandas as pd

In [27]:
df = pd.read_csv('/content/train.csv')

In [28]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [29]:
###Drop Nan Values
df=df.dropna()

In [30]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [31]:
## Get the Dependent features
y=df['label']

In [32]:
y.value_counts()

,count
label,
0,10361
1,7924


In [33]:
X.shape

(18285, 4)

In [34]:
y.shape

(18285,)

In [35]:
import tensorflow as tf

In [49]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN, Dropout

In [37]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [38]:
messages=X.copy()

In [39]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [40]:
messages.reset_index(inplace=True)

In [41]:
import nltk
import re
from nltk.corpus import stopwords

In [42]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [43]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [44]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [45]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[1898, 2839, 4730, 4427, 2862, 1248, 1441, 227, 4957, 4723],
 [859, 3419, 1263, 4236, 2303, 3648, 692],
 [3272, 722, 3770, 1233],
 [1272, 165, 4351, 4966, 357, 4235],
 [1650, 2303, 1064, 161, 865, 2056, 2303, 1039, 2370, 2880],
 [2123,
  1362,
  317,
  3422,
  2932,
  559,
  3017,
  4853,
  28,
  2221,
  4237,
  392,
  1451,
  2337,
  692],
 [4852, 243, 2578, 283, 4134, 2030, 2874, 3272, 4613, 2509, 551],
 [3463, 4515, 3243, 1980, 3021, 2435, 559, 139, 4613, 2509, 551],
 [1785, 2243, 907, 4207, 2740, 2398, 2858, 889, 559, 3428],
 [103, 669, 1838, 475, 528, 233, 1901, 4304],
 [3342, 4927, 1177, 674, 4478, 1520, 3612, 307, 4000, 83, 3343],
 [4966, 662, 2862, 2398, 559, 3021],
 [4305, 2320, 995, 2708, 4037, 381, 2725, 2337, 1661],
 [906, 1611, 23, 91, 3994, 4282, 4169, 4613, 2509, 551],
 [4043, 2954, 3593, 1973, 543, 4613, 2509, 551],
 [2459, 2454, 1143, 863, 2717, 2576, 1, 893, 178, 1960],
 [3164, 3327, 3419],
 [3713, 3109, 378, 1597, 559, 19, 2095, 692],
 [1922, 3344, 1263, 3096, 4151,

**Embedding Representation**

In [46]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  227 4957 4723]
 [   0    0    0 ... 2303 3648  692]
 [   0    0    0 ...  722 3770 1233]
 ...
 [   0    0    0 ... 4613 2509  551]
 [   0    0    0 ... 4781 1585 1486]
 [   0    0    0 ...   73 4322   27]]


In [47]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1898,
       2839, 4730, 4427, 2862, 1248, 1441,  227, 4957, 4723], dtype=int32)

## **Build the RNN model**

In [50]:
model = Sequential()

# Embedding layer (maps each word to a dense vector of fixed size)
model.add(Embedding(input_dim=5000, output_dim=128, input_length=X.shape[1]))

# Add a SimpleRNN layer (Recurrent Neural Network)
model.add(SimpleRNN(128, return_sequences=False))

# Add a Dropout layer to avoid overfitting
model.add(Dropout(0.2))

# Output layer (for binary classification)
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Summary of the model
print(model.summary())


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [51]:
len(embedded_docs),y.shape

(18285, (18285,))

In [52]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [53]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

**Model Training**

In [55]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 28ms/step - accuracy: 0.8277 - loss: 0.3485 - val_accuracy: 0.9132 - val_loss: 0.1985
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.9699 - loss: 0.0908 - val_accuracy: 0.9123 - val_loss: 0.2222
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.9953 - loss: 0.0192 - val_accuracy: 0.9039 - val_loss: 0.2965
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - accuracy: 0.9990 - loss: 0.0055 - val_accuracy: 0.9046 - val_loss: 0.3604
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 1.0000 - loss: 0.0017 - val_accuracy: 0.9074 - val_loss: 0.3709
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9999 - loss: 7.4314e-04 - val_accuracy: 0.9089 - val_loss: 0.4049
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 1.0000 - loss: 2.3208e-04 - val_accuracy: 0.9120 - val_loss: 0.4158
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 1.0000 - loss: 2.3761e-0

**Performance Metrics And Accuracy**

In [ ]:
##Performance Metrics And Accuracy

In [58]:
batch_size = 64
score, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print(f"Test Score: {score}")
print(f"Test Accuracy: {acc}")

95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9043 - loss: 0.5017
Test Score: 0.46671050786972046
Test Accuracy: 0.9101905822753906


In [59]:
from sklearn.metrics import confusion_matrix

In [60]:
y_pred=model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [ ]:
# Assuming y_pred contains continuous values (e.g., probabilities from a sigmoid output)
y_pred_binary = [1 if p > 0.5 else 0 for p in y_pred]

In [64]:
confusion_matrix(y_test,y_pred_binary)

array([[3070,  349],
       [ 193, 2423]])

In [63]:
# Assuming y_pred contains continuous values (e.g., probabilities from a sigmoid output)
y_pred_binary = [1 if p > 0.5 else 0 for p in y_pred]

# Now calculate accuracy using the binary predictions
accuracy = accuracy_score(y_test, y_pred_binary)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9101905550952776
